In [56]:
import pandas as pd
import numpy as np

## LOAD DATASETS


In [ ]:
sales_data = pd.read_csv('../data/sales_train.csv')
items = pd.read_csv('../data/items.csv')
item_categories = pd.read_csv('../data/item_categories.csv')
shops = pd.read_csv('../data/shops.csv')
test_data = pd.read_csv('../data/test.csv')

## EXPLORE DATA


### print all data


In [12]:
sales_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [14]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [15]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [16]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [17]:
test_data.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


### Check info data type **SALES_DATA**


In [19]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


## CLEANING DATASETS


Check missing value


In [23]:
print(sales_data.isnull().sum())

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64


Change data type **item_price** & **item_cnt_day** to **int**


In [29]:
sales_data['item_price'] = sales_data['item_price'].astype(int)
sales_data['item_cnt_day'] = sales_data['item_cnt_day'].astype(int)

print(sales_data.dtypes)

date              object
date_block_num     int64
shop_id            int64
item_id            int64
item_price         int32
item_cnt_day       int32
dtype: object


Remove outlier


In [32]:
sales_data = sales_data[sales_data['item_price'] > 0]
sales_data = sales_data[sales_data['item_cnt_day'] > 0]

Format data type **date** & change to **datetime[ns]**


In [42]:
sales_data['date'] = pd.to_datetime(sales_data['date'], format="%d.%M.%Y")

## FEATURE ENGINEERING


Make new column:

- month
- year


In [44]:
sales_data['month'] = sales_data['date'].dt.month
sales_data['year'] = sales_data['date'].dt.year

Group data for make new column: **item_cnt_month**


In [ ]:
monthly_data = sales_data.groupby(
    ['date_block_num', 'shop_id', 'item_id'],
    as_index=False
).agg({
    'item_cnt_day': 'sum'
}).rename(columns={
    'item_cnt_day' : 'item_cnt_month'
})

monthly_data.head()

,date_block_num,shop_id,item_id,item_cnt_month
1606595,33,59,22087,6
1606596,33,59,22088,2
1606597,33,59,22091,1
1606598,33,59,22100,1
1606599,33,59,22102,1


## MERGING DATA


Merge data sales with items


In [54]:
merged_data = monthly_data.merge(items, on='item_id', how='left')
merged_data = merged_data.merge(shops, on='shop_id', how='left')
merged_data = merged_data.merge(item_categories, on='item_category_id', how='left')

## SAVE DATA TO NEW FILE


In [55]:
merged_data.to_csv("../data/preprocced_sales_data.csv", index=False)